### 05. using tools with llms

- ge en modell tillgång till funktioner för att lösa problem, exempelvis matematik, privat data, dagsfärsk data

In [34]:
import os
from datetime import datetime
import json

from dotenv import load_dotenv
#from pydantic import BaseModel

from google import genai # client for google gemini api
from google.genai import types
import httpx

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY") # load the api key from another file (.env)

client = genai.Client(api_key=API_KEY) # create an instance of the client

In [28]:
promt = f"What is the current ({datetime.today()}) temperature in Amsterdam?"

In [29]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=promt,
)

print(response.candidates[0].content.parts[0].text)

I am unable to provide you with the current temperature in Amsterdam. I do not have real-time access to weather data.

To find the current temperature, I recommend using a reliable weather app or website like:

*   **Google Weather:** Search "weather in Amsterdam" on Google.
*   **AccuWeather:** [https://www.accuweather.com/](https://www.accuweather.com/)
*   **The Weather Channel:** [https://weather.com/](https://weather.com/)
*   **Local Dutch weather services (e.g., KNMI):** If you know of any local Dutch weather services, they might provide more accurate, localized data.

These sources should give you the most up-to-date information.



In [35]:
def get_location_coordinates(location: str) -> dict:
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={API_KEY}"

    response = httpx.get(url)
    result = json.loads(response.text)
    location = result["results"][0]["geometry"]["location"]

    return {"latitude": location["lat"], "longitude": location["lng"]}

In [30]:
def get_current_temperature(latitude: str, longitude: str) -> str:

    url = f"https://weather.googleapis.com/v1/currentConditions:lookup?key={API_KEY}&location.latitude={latitude}&location.longitude={longitude}"
    response = httpx.get(url)
    result = json.loads(response.text)

    return None


In [31]:
get_current_temperature_tool = {
    "name": "get_current_temperature",
    "description": "returns the current temperature for a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "name of location"},
        },
        "required": ["location"]
    },
}

In [32]:
temp_tool = types.Tool(function_declarations=[get_current_temperature_tool])
config = types.GenerateContentConfig(tools=[temp_tool])

In [33]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=promt,
    config=config
)

#print(response.candidates[0].content.parts[0].text)

print(response)


candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'location': 'Amsterdam'}, name='get_current_temperature'), function_response=None, text=None)], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=None, avg_logprobs=1.9220169633626938e-06, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)] create_time=None response_id=None model_version='gemini-2.0-flash' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=7, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=7)], prompt_token_count=56, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEX